In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [8]:
from twoDsampler import twoDsamplefromGaussian as tDsfG

ImportError: cannot import name 'twoDsamplefromGaussian' from 'twoDsampler' (/Users/phiamorton/Repos/Metropolis-Hastings/Metropolis-Hastings/twoDsampler.py)